### Done
* Describe Keras
* Show how to create a model in Keras

### To do
* Describe the 3 different ways of creating a model
* Save and load weights
* Accessing various parts of the sequential model
* Load pre-trained weights

### Notes


# Keras Fundamentals
**NB:** This notebook needs to download some material. Click `'Kernel'` >> `'Restar & Run All'` now, so that it can download while you read the introduction. (When download is complete click `'Kernel'` >> `'Restar & Clear All'`, and proceed as normal)

This notebook serves as a quick introduction to Keras.
As with the TF introduction this is very quick, so have a look at the **external resources** in order to get more depth:
* [Keras code](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/keras) in the TF github repo. Looking at the code is often the best way to understand what is going on.
* [Keras as a simplified interface to TensorFlow: tutorial](https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)
* [TensorFlow High-Level APIs: Models in a Box (TensorFlow Dev Summit 2017)](https://www.youtube.com/watch?v=t64ortpgS-E) 17 min video
* [Integrating Keras & TensorFlow: The Keras workflow, expanded (TensorFlow Dev Summit 2017)](https://www.youtube.com/watch?v=UeheTiBJ0Io) 18 min video


## High-level APIs
Pure TF is very verbose, and it is therefore a good idea to use a high-level API of some sort.
This simplifies and speeds-up development, reduces the risk of bugs, and generally reduces headache.
Another neat benefit is that a lot of best practices (initialization, scoping, etc.) are hard-coded into the functions.

There are other high-level APIs than Keras, such as 
[TFLearn](http://tflearn.org/),
[tf.learn](https://www.tensorflow.org/get_started/tflearn) (yes, the names are confusing :( ),
[tf.slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim), and
[tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers).
These are generally intercompatible, and offer overlaping functionality.
This can be confusing.
In order to simplify things we have chosen to focus on just one: Keras.

**[Keras](https://keras.io/)**
is a high-level API that can use TensorFlow or Theano as backend.
In early 2017 [TensorFlow chose Keras](http://www.fast.ai/2017/01/03/keras/) as the first high-level API to include into the tensorflow core (as opposed to contrib).
Keras has official Google support and has a large community and pre-existing examples.
Making it the best high-level API to learn at the moment (according to the author).

Keras sits of top of TFLayers, and shares its implementation (e.g. `tf.layers.dense` and `keras.layers.Dense` are the same).
Keras helps with model creation, training, and evaluation.
We won't use Keras to its full potential in this course, as it sometimes encapsulates the details to such an extent that it hinders learning.

![](keras_overview.png)



## A Basic Example

Keras can be accessed either as a stand alone library (that can use different backends), or through TF.
This only affects the import statements, and porting code back and forth should be very simple.
We will access Keras through TF, in order to ensure that the versions are fully compatible.


In [2]:
import os
import sys
sys.path.append(os.path.join('.', '..')) 
import utils

import tensorflow as tf
import tensorflow.contrib.keras as keras
from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras.api.keras.models import Sequential, Model
from tensorflow.contrib.keras.api.keras.layers import Dense, Input
from tensorflow.contrib.keras.api.keras.losses import categorical_crossentropy
from tensorflow.contrib.keras.api.keras.metrics \
    import categorical_accuracy

if int(tf.__version__[0]) < 1:
    print('WARNING: This will probably not work with TF version < 1.0')
    print('Your version is ' + tf.__version__)

### Multiple ways of solving MNIST

Keras kan be used in several ways

In [ ]:
## Create training loop
max_epoch = 101
batch_size = 64
def train(train_op, acc_value):
    init_op = tf.global_variables_initializer()
    with tf.Session().as_default() as sess:
        # Tell Keras to use the session to initialize all variables that it 
        # creates internally
        K.set_session(sess)
        sess.run(init_op)

        for i in range(max_epoch):
            batch = mnist_data.train.next_batch(batch_size)
            train_op.run(feed_dict={img:batch[0], labels:batch[1]})
            if i % 10 == 0:
                accuracy = acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels})
                print ('{:3} : {:.3}'.format(i, accuracy))
    graph = utils.rename_nodes(sess.graph_def, lambda s:"/".join(s.split('_',1)))
    return graph


In [ ]:

# Load data (download if you haven't already)
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
method = 3
print('Method ' + str(method))

# Reset the graph
tf.reset_default_graph()

# Prepare placeholders
img = tf.placeholder(tf.float32, shape=(None, 784), name='img')
labels = tf.placeholder(tf.float32, shape=(None, 10), name='label')

if method == 1:
    # Easy
    # Saving weights is hard
    # A lot of control
    with tf.name_scope('model'):
        x = Dense(128, activation='relu', name='Dense1')(img)
        x = Dense(128, activation='relu', name='Dense2')(x)
    preds = Dense(10, activation='softmax', name='prediction')(x)

elif method == 2:
    # Easy
    # saving weights is easy
    # Loss of control (ugly TB graph)
    with tf.name_scope('model'):
        model = Sequential()
        model.add(Dense(128, activation='relu', name='Dense1', input_dim=784)) 
        model.add(Dense(128, activation='relu', name='Dense2'))
    model.add(Dense(10, activation='softmax', name='prediction'))
    preds = model(img)
    
elif method == 3:
    # A bit less easy
    # Saving weights is easy
    # A lot of control    
    # Essentially method 1, wraped in a a model (similar to method 2)
        with tf.name_scope('model'):
            input_layer = Input(tensor=img)
            x = Dense(128, activation='relu', name='Dense1')(input_layer)
            x = Dense(128, activation='relu', name='Dense2')(x)
        x = Dense(10, activation='softmax', name='prediction')(x)

        model = Model(inputs=input_layer, outputs=x, name='asdf')
        preds = model.output
else:
    print('{} is not a valid method argument!'.format(method))

    
## Create ops
with tf.name_scope('accuracy'):
    acc_value = K.mean(categorical_accuracy(labels, preds))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(0.5)          
    train_op = optimizer.minimize(loss)

## Run training loop, and return the graph
graph = train(train_op, acc_value)

## Launch TensorBoard, and visualize the TF graph
utils.show_graph(graph)

## Using pre-trained models


One of the cool things about Keras is that it makes it very easy to use pretrained models!

Read more about [using pretrained models with Keras](https://keras.io/applications/)

22 million parameters

[Inception in TF](https://github.com/tensorflow/models/tree/master/inception)

Code: [inception_v3.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/keras/python/keras/applications/inception_v3.py). How it is implemented in Keras.

In [3]:
## RUN THIS CELL ASAP! It takes a while to complete!
## The cell downloads the weights (~80 mb), if they aren't already.

# Create a 
inception_stem = tf.contrib.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False, # Don't include the dense layers on top of the network
    input_shape=None, # width and height should be no smaller than 139
#     pool='avg'
)

# Initially we don't want to change 
inception_stem.trainable = False

In [ ]:
## Try and 
# inception_stem.summary()

# Credits
Created by Toke Faurby ([faur](https://github.com/Faur)).
